In [10]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline 
from scipy.integrate import cumtrapz
def getpoly(n,N=1e4, G=1, imax=1e4,tolerance = 1.e-19,show=False): 
    ''' Getpoly(n,,N=1e4, G=1, imax=1e4,tolerance = 1.e-19) returns (r,rho,m,P) for a self-consistent polytrope 
    with total mass and radius M=R=1.  Scale according to rho ~ M/R^3 and P ~ GM^2/R^4. ''' 
    N=int(N) # make sure N is an integer
    imax = int(imax) # make sure imax is an integer
    r = np.linspace(1.e-10/N,1,N) 
    rho = r.copy()*0.+4*np.pi/3;  
    rhoprev = rho +0.1
    
    keepgoing = 1 
    i = 0 
    while keepgoing: 
        m = cumtrapz(4*np.pi*r**2*rho,r,initial=0) # Could switch to Simpson interator for even greater accuracy

        mmax = np.max(m) ## normalize density and mass to unit mass. 
        rho = rho/mmax
        m = m/mmax

        w = ~np.isnan(rho/rhoprev) # necessary because the last value of rho is zero. 
        error = np.max(np.abs(rho[w]/rhoprev[w]-1))
        keepgoing = (error > tolerance)*(i<imax)

        if show: 
            plt.figure(2)
            if i: plt.plot(r,rhoprev,'r',r,rho,'k')

        g = -G*m/r**2
        Phi = cumtrapz(-g, r,initial=0)
        rhoprev = rho 
        rho = (np.max(Phi)-Phi)**n
        i +=1
    m = cumtrapz(4*np.pi*r**2*rho,r,initial=0) # Could switch to Simpson interator for even greater accuracy
    mmax = np.max(m) ## normalize density and mass to unit mass. 
    rho = rho/mmax
    m = m/mmax

    P = rho**(1.+1./n) ## un-normalized pressure profile. 
    # Normalize the pressure: 
    Egrav = np.trapz(G*m/r * rho *4*np.pi*r**2,r)
    TwoEtherm = 3*np.trapz(P*4*np.pi*r**2, r)
    P = P*Egrav/TwoEtherm ## enforce virial equilibrium: |E_grav| = 2 E_therm 
    
    return r,rho,m, P

def getpoly_phys(n, R_phys, M_phys, G_phys, **kwargs):
    """
    Compute dimensionless polytrope (R=1, M=1), then
    rescale to physical units R_phys, M_phys, G_phys.
    Any kwargs (N, imax, tolerance, show) pass to getpoly.
    """
    # 1) get dimensionless solution
    r0, rho0, m0, P0 = getpoly(n, G=1.0, **kwargs)

    # 2) scale radius & enclosed mass
    r_phys = r0 * R_phys
    m_phys = m0 * M_phys

    # 3) density scales as M / R^3
    rho_phys = rho0 * (M_phys / R_phys**3)

    # 4) pressure scales as G M^2 / R^4
    P_phys = P0 * (G_phys * M_phys**2 / R_phys**4)

    return r_phys, rho_phys, m_phys, P_phys

In [11]:
# Pick your polytropic index here:
n = 3

# Generate the profile
r, rho, m, P = getpoly_phys(n,R_phys=20.0, M_phys=20.0, G_phys=20.0, show=False)

# Build a DataFrame and write CSV
df = pd.DataFrame({"r": r, "rho": rho, "m": m, "P": P})
csvname = f"20polytrope_n{n:.2f}.csv"
df.to_csv(csvname, index=False)
print(f"Saved {len(r)} points to {csvname!r}")

/var/folders/wb/fw1jvhk90ld87h224nllm79h0000gn/T/ipykernel_44444/3896489675.py:18: DeprecationWarning: 'scipy.integrate.cumtrapz' is deprecated in favour of 'scipy.integrate.cumulative_trapezoid' and will be removed in SciPy 1.14.0
  m = cumtrapz(4*np.pi*r**2*rho,r,initial=0) # Could switch to Simpson interator for even greater accuracy
/var/folders/wb/fw1jvhk90ld87h224nllm79h0000gn/T/ipykernel_44444/3896489675.py:33: DeprecationWarning: 'scipy.integrate.cumtrapz' is deprecated in favour of 'scipy.integrate.cumulative_trapezoid' and will be removed in SciPy 1.14.0
  Phi = cumtrapz(-g, r,initial=0)
/var/folders/wb/fw1jvhk90ld87h224nllm79h0000gn/T/ipykernel_44444/3896489675.py:24: RuntimeWarning: invalid value encountered in divide
  w = ~np.isnan(rho/rhoprev) # necessary because the last value of rho is zero.
/var/folders/wb/fw1jvhk90ld87h224nllm79h0000gn/T/ipykernel_44444/3896489675.py:37: DeprecationWarning: 'scipy.integrate.cumtrapz' is deprecated in favour of 'scipy.integrate.cumula

Saved 10000 points to '20polytrope_n3.00.csv'
